In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
%env TORCH_USE_CUDA_DSA=True

import torch

from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM

import sys
sys.path.append("/home/blacksamorez/quip-sharp")
from lib.utils.unsafe_import import model_from_hf_path
from model.llama import LlamaForCausalLM as QuipSharpLlamaForCausalLM

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
env: TORCH_USE_CUDA_DSA=True


I0130 00:36:03.164406 594052 utils.py:145] Note: detected 255 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
I0130 00:36:03.166201 594052 utils.py:148] Note: NumExpr detected 255 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
I0130 00:36:03.166588 594052 utils.py:160] NumExpr defaulting to 8 threads.
I0130 00:36:03.450076 594052 config.py:58] PyTorch version 2.1.2 available.
Using `is_flash_attn_available` is deprecated and will be removed in v4.38. Please use `is_flash_attn_2_available` instead.
Using `is_flash_attn_available` is deprecated and will be removed in v4.38. Please use `is_flash_attn_2_available` instead.
Using `is_flash_attn_available` is deprecated and will be removed in v4.38. Please use `is_flash_attn_2_available` instead.


In [2]:
aqlm_model = AutoModelForCausalLM.from_pretrained("BlackSamorez/Llama-2-7b-AQLM-6288ppl-hf", trust_remote_code=True, low_cpu_mem_usage=True, torch_dtype="auto").cuda()
quip_sharp_model = model_from_hf_path(
    "relaxml/Llama-2-7b-E8P-2Bit",
    use_flash_attn=True,
    use_cuda_graph=False,
)[0].cuda()
fp16_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", torch_dtype="auto").cuda()
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

W0130 00:36:54.963166 594052 warnings.py:109] /home/blacksamorez/quip-sharp/.conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
I0130 00:37:13.597410 594052 modeling.py:920] We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
output = aqlm_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), max_new_tokens=30)
print("AQLM:\n", tokenizer.decode(output[0]), "\n")
output = quip_sharp_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), max_new_tokens=30)
print("QUIP#:\n", tokenizer.decode(output[0]), "\n")
output = fp16_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), max_new_tokens=30)
print("FP16:\n", tokenizer.decode(output[0]), "\n")

AQLM:
 <s> Hi, I'm a newbie here. I'm looking for a good place to learn about the basics of programming.
I'm 

QUIP#:
 <s> Hi! I'm looking for a new and exciting adventure in my life. hopefully in a few days I will have the chance to join you 

FP16:
 <s> Hi everyone, I'm new to the forum.
I've been playing with a few different synths lately and I've been having 



In [4]:
%%time
output = aqlm_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), min_new_tokens=200, max_new_tokens=200)

CPU times: user 8.64 s, sys: 22.9 ms, total: 8.66 s
Wall time: 8.65 s


In [5]:
%%time
output = quip_sharp_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), min_new_tokens=200, max_new_tokens=200)

CPU times: user 15.2 s, sys: 29.2 s, total: 44.4 s
Wall time: 45.9 s


In [6]:
%%time
output = fp16_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), min_new_tokens=200, max_new_tokens=200)

CPU times: user 5.94 s, sys: 5.65 ms, total: 5.95 s
Wall time: 5.94 s
